Analyze zuora?


In [ ]:
var request = require('request');
var fs = require('fs');
var { URL } = require('url');

var zuoraOauth = (credentials) => {
    var rest_api, rest_user, rest_pass;
}

var httpRequest = (options) => {
    return new Promise((resolve, reject) => {
        request(options, function (error, res, body) {
            if(error) {
                reject(error);
            }
            resolve(body);
        });
        
        if(typeof body !== 'undefined') {
            req.write(body);
        }
    });
}

var token, expires;
var eloquaOauth = (eloquaConfig) => {
    if(typeof eloquaConfig === 'undefined'
       || typeof eloquaConfig.rest_api_company === 'undefined'
       || typeof eloquaConfig.rest_api_user === 'undefined'
       || typeof eloquaConfig.rest_api_password === 'undefined'
       || typeof eloquaConfig.rest_client_id === 'undefined'
       || typeof eloquaConfig.rest_secret === 'undefined') {
        throw new Error('Please supply valid config eloqua configuration.');
    }
    var authBody = {
        "grant_type": "password",
        "scope": "full",
        "username": eloquaConfig.rest_api_company + '\\' + eloquaConfig.rest_api_user,
        "password": eloquaConfig.rest_api_password
    };
    return httpRequest({
        followAllRedirects: true,
        uri: eloquaConfig.token_uri,
        method: 'POST',
        json: authBody,
        headers: {
            'Authorization': "Basic " + new Buffer(eloquaConfig.rest_client_id + ":" + eloquaConfig.rest_secret).toString("base64")
        },
    })
        .then(newToken => {
            token = newToken.access_token;
            newToken['expires'] = expires = new Date((new Date()).getTime() + parseFloat(newToken.expires_in) * 1000);
            return newToken;
        })
}

var zuoraExport = (query) => {
    
}

var csvToJson = (csv) => {
    
}

var mapDataToFields = () => {
    
}

var eloquaBulkImport = (json) => {
    
}

module.exports = {
    eloquaBulkImport,
    mapDataToFields,
    csvToJson,
    zuoraExport,
    zuoraOauth,
    eloquaOauth
}


Test zuora?


In [1]:
var assert = require('assert');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

describe('zuora to eloqua', () => {
    beforeEach(() => {
        var xlsx = require('xlsx');
        var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
        var worksheet = PROFILE_PATH + '/Documents/asm/Marketing_File_Oct_.xslx';
        var zuoraConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/zuoraRest_production.json').toString());

    })
    
    it('should transfer data end-to-end', () => {
        var success = eloquaBulkImport(
            mapDataToFields(
                csvToJson(
                    zuoraExport(
                        zuoraOauth()))), eloquaOauth());
        assert(success);
    })
    
    it('should have comparable records', () => {
        const workSheetsFromFile = XLSX.utils.sheet_to_json(worksheet);
        console.log(workSheetsFromFile);
    })
})

describe('zuora oauth', () => {
    it('should connect to zuora using oauth', () => {
        
    })
})

describe('zuora export', () => {
    it('should export a CSV from zuora', () => {
        
    })
    
    it('should convert CSV to JSON', () => {
        
    })
})

describe('map zuora data fields', () => {
    it('should map basic data', () => {
        
    })
    
    it('should map contact data', () => {
        
    })
    
    it('should map support data', () => {
        
    })
    
    it('should map cancelled data', () => {
        
    })
})

describe('eloqua bulk upload', () => {
    var eloquaToken, eloquaConfig;
    
    beforeEach(() => {
        try {
            eloquaConfig = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_production.json').toString().trim());
            eloquaToken = JSON.parse(fs.readFileSync(PROFILE_PATH + '/.credentials/eloqua_token.json').toString().trim());
        } catch (e) {
        }
    })
    
    it('should authenticate using eloqua oauth', (done) => {
        Promise.resolve(typeof eloquaToken !== 'undefined'
                    ? eloquaToken
                    : eloquaOauth(eloquaConfig))
            .then(r => {
                fs.writeFileSync(
                        PROFILE_PATH + '/.credentials/eloqua_token.json',
                        JSON.stringify(r, null, 4));
                assert((new Date(r.expires)).getTime() > (new Date()).getTime());
                done()
            })
    })
    
    it('should update data to eloqua', () => {
        
    })
    
    it('should confirm upload was successful', () => {
        
    })
})


ReferenceError: describe is not defined

In [1]:
var importer = require('../Core');
var path = require('path');
var module = require('module');

// call test runner
var Mocha = require('mocha');
var mocha = new Mocha({
    ui: 'tdd',
    reporter: 'list'
});

var importScript = (r) => {
    const ctx = {};
    Object.assign(ctx, zuora)
    mocha.suite.emit('pre-require', ctx, r.id, mocha);
    Object.assign(ctx, Mocha)
    mocha.suite.emit('require', r.runInNewContext(ctx), r.id, mocha);
    mocha.suite.emit('post-require', ctx, r.id, mocha);    
}

$$.async();
var oldDir = process.cwd();
var ctx = {}, zuora;
importer.import('analyze zuora')
    .then(r => {
        zuora = r;
        return importer.interpret('test zuora')
    })
    .then(r => {
        const f = 'C:\\Users\\brian.cullinan\\Documents\\Selenium\\ecommerce\\test\\specs\\trial.js'
        process.chdir(path.dirname(f));
        
        const tmp = {
            id: f,
            runInNewContext: (ctx) => {
                ctx.__filename = f;
                ctx.useCache = false;
                //return importer.runInNewContext('require(' + JSON.stringify(f) + ')', ctx, {}, false)
                return importer.runInNewContext(fs.readFileSync(f).toString(), ctx, {})
            }
        }
        return importScript(tmp);
    })
    .then(() => new Promise(resolve => mocha.run(function (failures) {
        process.chdir(oldDir);
        resolve(failures);
    })))
    .then(r => $$.sendResult())
    .catch(e => $$.sendError(e))


undefined


ReferenceError: describe is not defined

Zuora query?


In [ ]:
var excludedProducts = [
    'Act! Pro - New License',
    'Act! Pro - 30 Day Support',
    'Act! Pro - Upgrade License',
    'Act! Password Reset Charge',
    'Act! Premium Cloud - Trial',
    'Act! Pro V19 - Upgrade License',
]
var excludedSkus = [
    '00000006'
]
var currencies = [
    'USD',
    'AUD',
    'NZD',
]


/*
GET_MODIFIED_SUBSCRIPTIONS,
from.ToString("yyyy-MM-dd"),
to.ToString("yyyy-MM-dd"),
DateTime.Now.ToString("yyyy-MM-dd"),
string.Join("' AND ProductRatePlan.Name!='", renewalsServiceConfiguration.ProductExclusions),
string.Join("' AND Product.SKU!='", renewalsServiceConfiguration.SkuExclusions),
string.Join("' OR Account.Currency='", renewalsServiceConfiguration.CurrencyInclusions)
*/

module.exports = `SELECT
    Account.Id,
    Account.Name,
    Account.AccountNumber,
    Account.resellerofRecord__c,
    Account.renewalRep__c,
    Account.commisionedSalesRep__c,
    Account.CreatedDate,
    SoldToContact.WorkEmail,
    SoldToContact.Country,
    BillToContact.WorkEmail,
    RatePlanCharge.Id,
    RatePlanCharge.BillingPeriod,
    RatePlanCharge.Description,
    RatePlanCharge.Quantity,
    RatePlanCharge.Version,
    RatePlanCharge.CreatedDate,
    RatePlanCharge.EffectiveEndDate,
    ProductRatePlan.planType__c,
    ProductRatePlan.planSubType__c,
    Product.productType__c,
    Subscription.Name,
    Subscription.Status,
    Subscription.Reseller__c,
    Subscription.SubscriptionEndDate,
    Subscription.SubscriptionStartDate,
    Subscription.TermStartDate,
    Subscription.TermEndDate,
    Subscription.AutoRenew
FROM RatePlanCharge
WHERE Subscription.Status!='Draft' AND Subscription.Status!='Expired'
    AND Subscription.TermEndDate &gt;='{0}' AND Subscription.TermEndDate &lt;='{1}'
    AND (Account.Currency='{5}')
    AND (Product.SKU!='{4}')
    AND (RatePlanCharge.EffectiveEndDate &gt;='{2}' OR RatePlanCharge.ChargeType='OneTime')
    AND (ProductRatePlan.Name!='{3}')
    AND RatePlanCharge.BillingPeriod!='Month'`
.replace('{3}', excludedRatePlans)
.replace('{4}', excludedProductSkus)
.replace('{5}', currencies)
